### Stations

In [1]:
import pandas as pd


stations = pd.read_excel('./spreadsheets/secoora_station_assets.xlsx', index_col=2)

### HF-radar

In [2]:
hfradar = pd.read_excel('./spreadsheets/secoora_hfradar_sites.xlsx', index_col=3)
ranges = dict({'16': 100, '12': 130, '8': 160, '5': 190})
hfradar['Range'] = [ranges['{}'.format(int(key))] for key in  hfradar['MHz']]

### Save matplotlib patch as GeoJSON

In [3]:
from matplotlib.patches import Wedge


def make_wedge(radar):
    """
    Make a HF-radar "wedge" from a StartAngle, SpreadAngle, Center (Longitude, Latitude), and Radius (radar range).

    """
    r = radar['Range'] / 111.1  # deg2km
    theta1, theta2 = radar['StartAngle'], radar['SpreadAngle'],
    center = radar['Longitude'], radar['Latitude']
    kw = dict(alpha=0.3, color='lightgreen')
    return Wedge(center, r, theta1+theta2, theta1, **kw)

In [4]:
import json
from shapely.geometry import polygon, mapping

for name, radar in hfradar.dropna().iterrows():
    patch = make_wedge(radar)
    
    p = patch.get_path()
    shape = polygon.Polygon(p.vertices)
    
    with open('hfradar/{}.geojson'.format(name), "w") as geojson:
        data = json.dumps(mapping(shape), indent=2)
        geojson.write(data)

This should be implemented as part of folium's API. For now the workaround is a "hackish" hard-coded icon insert.

In [5]:
def normalize_name(name):
    name = name.strip().lower()
    name = name.replace("'", "")
    name = name.replace(" ", "_")
    return name

In [6]:
green_hfradar = """
var hf_radar_{name} = L.icon({{
iconUrl: 'https://raw.githubusercontent.com/ioos/secoora/master/AbstractsPostersPresentations/assets_map/secoora_icons/hfradar-green.png',
iconSize: [14, 14]}});
L.marker({location}, {{icon: hf_radar_{name}}}).addTo(map).bindPopup("{popup}");
""".format

orange_hfradar = """
var hf_radar_{name} = L.icon({{
iconUrl: 'https://raw.githubusercontent.com/ioos/secoora/master/AbstractsPostersPresentations/assets_map/secoora_icons/hfradar-orange.png',
iconSize: [14, 14]}});
L.marker({location}, {{icon: hf_radar_{name}}}).addTo(map).bindPopup("{popup}");
""".format

yellow_hfradar = """
var hf_radar_{name} = L.icon({{
iconUrl: 'https://raw.githubusercontent.com/ioos/secoora/master/AbstractsPostersPresentations/assets_map/secoora_icons/hfradar-yellow.png',
iconSize: [14, 14]}});
L.marker({location}, {{icon: hf_radar_{name}}}).addTo(map).bindPopup("{popup}");
""".format

Stations

In [7]:
green_buoy = """
var buoy_{name} = L.icon({{
iconUrl: 'https://raw.githubusercontent.com/ioos/secoora/master/AbstractsPostersPresentations/assets_map/secoora_icons/buoy-green.png',
iconSize: [14, 14]}});
L.marker({location}, {{icon: buoy_{name}}}).addTo(map).bindPopup("{popup}");
""".format

In [8]:
green_coastal = """
var buoy_{name} = L.icon({{
iconUrl: 'https://raw.githubusercontent.com/ioos/secoora/master/AbstractsPostersPresentations/assets_map/secoora_icons/shore_station-green.png',
iconSize: [14, 14]}});
L.marker({location}, {{icon: buoy_{name}}}).addTo(map).bindPopup("{popup}");
""".format

orange_coastal  = """
var buoy_{name} = L.icon({{
iconUrl: 'https://raw.githubusercontent.com/ioos/secoora/master/AbstractsPostersPresentations/assets_map/secoora_icons/shore_station-orange.png',
iconSize: [14, 14]}});
L.marker({location}, {{icon: buoy_{name}}}).addTo(map).bindPopup("{popup}");
""".format

yellow_coastal  = """
var buoy_{name} = L.icon({{
iconUrl: 'https://raw.githubusercontent.com/ioos/secoora/master/AbstractsPostersPresentations/assets_map/secoora_icons/shore_station-yellow.png',
iconSize: [14, 14]}});
L.marker({location}, {{icon: buoy_{name}}}).addTo(map).bindPopup("{popup}");
""".format

In [9]:
green_bottom_mtn = """
var buoy_{name} = L.icon({{
iconUrl: 'https://raw.githubusercontent.com/ioos/secoora/master/AbstractsPostersPresentations/assets_map/secoora_icons/tri-green.png',
iconSize: [14, 14]}});
L.marker({location}, {{icon: buoy_{name}}}).addTo(map).bindPopup("{popup}");
""".format

orange_bottom_mtn = """
var buoy_{name} = L.icon({{
iconUrl: 'https://raw.githubusercontent.com/ioos/secoora/master/AbstractsPostersPresentations/assets_map/secoora_icons/tri-orange.png',
iconSize: [14, 14]}});
L.marker({location}, {{icon: buoy_{name}}}).addTo(map).bindPopup("{popup}");
""".format

In [10]:
green_bottom_stn = """
var buoy_{name} = L.icon({{
iconUrl: 'https://raw.githubusercontent.com/ioos/secoora/master/AbstractsPostersPresentations/assets_map/secoora_icons/circ-green.png',
iconSize: [12, 12]}});
L.marker({location}, {{icon: buoy_{name}}}).addTo(map).bindPopup("{popup}");
""".format

In [11]:
from glob import glob
import folium

lon, lat = -86.276, 30.935 
zoom_start = 5

mapa = folium.Map(location=[lat, lon], tiles="OpenStreetMap",
                  zoom_start=zoom_start)


jsons = glob('hfradar/*.geojson')
for json in jsons:
    kw = dict(fill_color='#DEFFDE', line_color='#AECCAE')
    mapa.geo_json(geo_path=json, **kw)


hfradars = []
for name, row in hfradar.iterrows():
    popup = '{} ({} MHz)'.format(row['DisplayTitle'], row['MHz'])
    name = normalize_name(name)
    location = [row['Latitude'], row['Longitude']]
    info = dict(name=name,
                popup=popup,
                location=location)
    status = row['Status']
    if status == 'Operational':
        hfradars.append(green_hfradar(**info))
    elif status == 'Permitting':
        hfradars.append(yellow_hfradar(**info))
    elif status == 'Planned':
        hfradars.append(orange_hfradar(**info))

platforms = []
for name, row in stations.iterrows():
    popup = name#.replace("'", "")
    name = normalize_name(name)
    location = [row['Latitude'], row['Longitude']]
    info = dict(name=name,
                popup=popup,
                location=location)
    
    platform_type, status = row['PlatformType'], row['Status']
    if  platform_type == 'Fixed Surface Buoy':  # All buoys are green for now.
        platforms.append(green_buoy(**info))
    elif platform_type == 'Fixed Coastal Station' and status == 'Operational':
        platforms.append(green_coastal(**info))
    elif platform_type == 'Fixed Coastal Station' and status == 'Construction':
        platforms.append(yellow_coastal(**info))
    elif platform_type == 'Fixed Coastal Station' and status == 'Planned':
        platforms.append(orange_coastal(**info))
    elif platform_type == 'Fixed Bottom Mount Mooring' and status == 'Operational':
        platforms.append(green_bottom_mtn(**info))
    elif platform_type == 'Fixed Bottom Mount Mooring' and status == 'Planned':
        platforms.append(orange_bottom_mtn(**info))
    elif platform_type == 'Fixed Bottom Station' and status == 'Operational':
        platforms.append(green_bottom_stn(**info))

fname = "secoora_assets.html"
mapa.create_map(fname)

In [12]:
import fileinput

def inline_map(mapa):
    from folium import Map
    from IPython.display import IFrame
    embed = IFrame(mapa, width=750, height=500)
    return embed


assets = '\n'.join(hfradars + platforms) 

add_lines = False
for line in fileinput.input(fname, inplace=True):
    if 'gJson_layer_13' in line:
        add_lines = True
    else:
        if add_lines:
            print(assets)
        add_lines = False
    print line,

inline_map(fname)